In [1]:
from datetime import datetime
import numpy as np

import dreamcoder as dc
from dreamcoder.frontier import Frontier, FrontierEntry
from dreamcoder.fragmentGrammar import FragmentGrammar
from dreamcoder.grammar import Grammar
from dreamcoder.program import Program
from dreamcoder.program import Abstraction
from dreamcoder.utilities import numberOfCPUs

import dreamcoder.domains.quantum_circuits.primitives as pr
from dreamcoder.domains.quantum_circuits.primitives import execute_program, get_qiskit_circuit, tcircuit, no_op
from dreamcoder.domains.quantum_circuits.tasks import QuantumTask


%load_ext line_profiler
%load_ext autoreload
%autoreload 2


In [2]:
n_qubit_tasks = 3
evaluationTimeout = 0.01

enum_settings = {
    "maximumFrontier": 1,
    
}
library_settings = {
    "structurePenalty": 6,  # increase regularization 3 4 (it was 1), look at a few in [1,15]
    "pseudoCounts": 10,  # increase to 100, test a few values
    "topK": 2,
    "arity": 3,
}

# Unfortunately these flags are set globally
dc.domains.quantum_circuits.primitives.GLOBAL_LIMITED_CONNECTIVITY = False
dc.domains.quantum_circuits.primitives.GLOBAL_NQUBIT_TASK = n_qubit_tasks 

In [3]:
def visualize_program(code: str):
    "Helper function to plot a circuit program"
    arguments = (*range(n_qubit_tasks), (n_qubit_tasks, ()))
    program = Program.parse(code)
    circuit = execute_program(program, arguments)
    reconstructed_circuit = get_qiskit_circuit(circuit)
    print(code)
    print(reconstructed_circuit.circuit)

In [4]:
primitives = [
    pr.p_hadamard,
    pr.p_t,
    pr.p_tdg,
    pr.p_cnot,
    pr.p_x,
]
grammar = Grammar.uniform(primitives) # continuationType=tcircuit not supported (non-unary argument)

In [15]:
programs = [
    "(lambda (lambda (lambda (lambda (cnot (x $0 $1) $1 $2)))))",
    "(lambda (lambda (lambda (lambda (cnot (x (x (x $0 $1) $1) $1) $1 $2)))))",
    "(lambda (lambda (lambda (lambda (x (x (x $0 $1) $1) $1) ))))",
    "(lambda (lambda (lambda (lambda (cnot (x (h (x $0 $2) $2) $2) $2 $1)))))",
    "(lambda (lambda (lambda (lambda (cnot (x (h (x (h $0 $2) $2) $2) $2) $2 $1)))))",
    "(lambda (lambda (lambda (lambda (cnot (x (h (x (h (x (h (x $0 $2) $2) $2) $2) $2) $2) $2) $2 $1)))))",
]
visualize_program(programs[1])

(lambda (lambda (lambda (lambda (cnot (x (x (x $0 $1) $1) $1) $1 $2)))))
     ┌───┐┌───┐┌───┐     
q_0: ┤ X ├┤ X ├┤ X ├──■──
     └───┘└───┘└───┘┌─┴─┐
q_1: ───────────────┤ X ├
                    └───┘
q_2: ────────────────────
                         


In [19]:
# Generate a few example tasks
solutions = {}  # dict of task:solution

for code in programs:
    arguments = (*range(n_qubit_tasks), (n_qubit_tasks, ()))
    program = Program.parse(code)
    circuit = execute_program(program, arguments)
    task = QuantumTask(f"t_{len(solutions):03d}_{code}", circuit)

    # 0 = solved, -inf = not solved
    likelihood = task.logLikelihood(program, evaluationTimeout)
    prior = grammar.logLikelihood(task.request, program)

    frontier_entry = FrontierEntry(
        program=program, logLikelihood=likelihood, logPrior=prior
    )

    solutions[task] = Frontier(
        frontier=[frontier_entry],  # multiple solutions are allowed
        task=task,
    )
tasks = list(solutions.keys())

In [20]:
generator = dc.dreamcoder.ecIterator(
    grammar=grammar,
    tasks=tasks,
    testingTasks=[],
    outputPrefix=f"experimentOutputs/quantum/{datetime.now().isoformat()}",
    enumerationTimeout=10,
    iterations=3,
    taskBatchSize=len(tasks),  # smaller should be faster
    taskReranker="randomShuffle",  # default
    CPUs=1,#numberOfCPUs(),
    solver="bottom",
    compressor="pypy",
    **enum_settings,
    **library_settings,
)

In [21]:
for result in generator:
    ...

dreamcoder.py:191 > Warning: Recognition model needs feature extractor. Ignoring recognition model.
dreamcoder.py:270 > Running EC on polymathiclin033.flatironinstitute.org @ 2024-08-20 08:35:48.819377 with 1 CPUs and parameters:
dreamcoder.py:273 > 	 noConsolidation  =  False
dreamcoder.py:273 > 	 iterations  =  3
dreamcoder.py:273 > 	 enumerationTimeout  =  10
dreamcoder.py:273 > 	 useRecognitionModel  =  False
dreamcoder.py:273 > 	 topk_use_only_likelihood  =  False
dreamcoder.py:273 > 	 pseudoCounts  =  10
dreamcoder.py:273 > 	 aic  =  1.0
dreamcoder.py:273 > 	 structurePenalty  =  6
dreamcoder.py:273 > 	 arity  =  3
dreamcoder.py:273 > 	 taskBatchSize  =  6
dreamcoder.py:273 > 	 taskReranker  =  randomShuffle
dreamcoder.py:273 > 	 storeTaskMetrics  =  False
dreamcoder.py:273 > 	 rewriteTaskMetrics  =  True
dreamcoder.py:273 > 	 maximumFrontier  =  1
dreamcoder.py:273 > 	 solver  =  bottom
dreamcoder.py:273 > 	 topK  =  2
dreamcoder.py:274 > 	 evaluationTimeout  =  1.0
dreamcoder.p

grammar.py:2048 >  -- Bottom up enumeration, cost 25
grammar.py:2048 >  -- Bottom up enumeration, cost 26
grammar.py:2048 >  -- Bottom up enumeration, cost 27
grammar.py:2048 >  -- Bottom up enumeration, cost 28
grammar.py:2048 >  -- Bottom up enumeration, cost 29
grammar.py:2048 >  -- Bottom up enumeration, cost 30
grammar.py:2048 >  -- Bottom up enumeration, cost 31
grammar.py:2048 >  -- Bottom up enumeration, cost 32
grammar.py:2048 >  -- Bottom up enumeration, cost 33
grammar.py:2048 >  -- Bottom up enumeration, cost 34
grammar.py:2048 >  -- Bottom up enumeration, cost 35
grammar.py:2048 >  -- Bottom up enumeration, cost 36
enumeration.py:446 > Enumerated 33140 programs
dreamcoder.py:565 > Generative model enumeration results:
dreamcoder.py:566 > HIT t_003_(lambda (lambda (lambda (lambda (cnot (x (h (x $0 $2) $2) $2) $2 $1))))) w/ (lambda (lambda (lambda (lambda (cnot (x (h (x $0 $2) $2) $2) $2 $1))))) ; log prior = -14.451859 ; log likelihood = 0.000000
HIT t_002_(lambda (lambda (

In [46]:
result.grammars[2].logLikelihood(tasks[1].request, Program.parse(programs[1]))

-13.89130210503095

In [48]:
tasks[0].logLikelihood(programs[0])

0.0

In [25]:
print(grammar)

print(result.grammars[-1])

0.000000	t0	$_
0.000000	tcircuit -> int -> tcircuit	h
0.000000	tcircuit -> int -> tcircuit	t
0.000000	tcircuit -> int -> tcircuit	tdg
0.000000	tcircuit -> int -> int -> tcircuit	cnot
0.000000	tcircuit -> int -> tcircuit	x
-0.092373	t0	$_
-0.587787	tcircuit -> int -> tcircuit	x
-0.656780	tcircuit -> int -> tcircuit	h
-0.656780	tcircuit -> int -> int -> tcircuit	cnot
-0.993252	tcircuit -> int -> tcircuit	t
-0.993252	tcircuit -> int -> tcircuit	tdg


In [29]:
frontiers = [f for f in solutions.values()]

new_grammar, new_frontiers = FragmentGrammar.induceFromFrontiers(
    g0=grammar, frontiers=frontiers, **library_settings
)

new_grammar, new_frontiers

0.000000	t0	$_
0.000000	tcircuit -> int -> tcircuit	h
0.000000	tcircuit -> int -> tcircuit	t
0.000000	tcircuit -> int -> tcircuit	tdg
0.000000	tcircuit -> int -> int -> tcircuit	cnot
0.000000	tcircuit -> int -> tcircuit	x
[Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (x $0 $1) $1 $2))))), logPrior=-8.671115273688494, logLikelihood=0.0], task=t_000_(lambda (lambda (lambda (lambda (cnot (x $0 $1) $1 $2)))))), Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (x (h (x $0 $2) $2) $2) $2 $1))))), logPrior=-14.451858789480823, logLikelihood=0.0], task=t_001_(lambda (lambda (lambda (lambda (cnot (x (h (x $0 $2) $2) $2) $2 $1)))))), Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (x (h (x (h $0 $2) $2) $2) $2) $2 $1))))), logPrior=-17.34223054737699, logLikelihood=0.0], task=t_002_(lambda (lambda (lambda (lambda (cnot (x (h (x (h $0 $2) $2) $2) $2) $2 $1)))))), Frontier(entries=[FrontierEntry(program=(la

fragmentGrammar.py:300 > Inducing a grammar from 4 frontiers
fragmentGrammar.py:329 > Starting score -98.33160637797658
fragmentGrammar.py:336 > Proposed 798 fragments.
fragmentGrammar.py:367 > New primitive of type tcircuit	(x (h $0 $1) $1)	
(score = -96.948606; dScore = 1.383000; <uses> = 5.828075)
fragmentGrammar.py:380 > 	(<uses> in rewritten frontiers: 6.000000)
fragmentGrammar.py:336 > Proposed 76 fragments.
fragmentGrammar.py:399 > Old joint = -66.478550	New joint = -48.853936

fragmentGrammar.py:419 > 0.000000 / 17.000000	h
fragmentGrammar.py:419 > 0.000000 / 17.000000	t
fragmentGrammar.py:419 > 0.000000 / 17.000000	tdg
fragmentGrammar.py:419 > 4.000000 / 17.000000	cnot
fragmentGrammar.py:419 > 3.000000 / 17.000000	x
fragmentGrammar.py:419 > 6.000000 / 17.000000	#(lambda (lambda (x (h $0 $1) $1)))


(<dreamcoder.grammar.Grammar at 0x7fa06076f310>,
 [Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (x $0 $1) $1 $2))))), logPrior=-8.618441027513024, logLikelihood=0.0], task=t_000_(lambda (lambda (lambda (lambda (cnot (x $0 $1) $1 $2)))))),
  Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (#(lambda (lambda (x (h $0 $1) $1))) $2 (x $0 $2)) $2 $1))))), logPrior=-11.546233809996629, logLikelihood=0.0], task=t_001_(lambda (lambda (lambda (lambda (cnot (x (h (x $0 $2) $2) $2) $2 $1)))))),
  Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (#(lambda (lambda (x (h $0 $1) $1))) $2 (#(lambda (lambda (x (h $0 $1) $1))) $2 $0)) $2 $1))))), logPrior=-11.376548837207187, logLikelihood=0.0], task=t_002_(lambda (lambda (lambda (lambda (cnot (x (h (x (h $0 $2) $2) $2) $2) $2 $1)))))),
  Frontier(entries=[FrontierEntry(program=(lambda (lambda (lambda (lambda (cnot (#(lambda (lambda (x (h $0 $1) $1))) $2 (#(lambda

In [16]:
type(new_grammar.primitives[-1])

dreamcoder.program.Invented

In [17]:
new_grammar.primitives[-1]

#(lambda (lambda (x (h $0 $1) $1)))

In [18]:
print(new_grammar)

-0.173272	t0	$_
-0.693147	tcircuit -> int -> tcircuit	x
-0.757686	tcircuit -> int -> int -> tcircuit	cnot
-1.163151	tcircuit -> int -> tcircuit	h
-1.163151	tcircuit -> int -> tcircuit	t
-1.163151	tcircuit -> int -> tcircuit	tdg
-0.693147	int -> tcircuit -> tcircuit	#(lambda (lambda (x (h $0 $1) $1)))


In [19]:
p = new_frontiers[0].entries[0].program

In [20]:
p.body.body.body.body

(cnot (x $0 $1) $1 $2)

### Defining a program

In [15]:
arguments = (*range(n_qubit_tasks), (n_qubit_tasks, ()))
program = Program.parse(code)
circuit = execute_program(program, arguments)
reconstructed_circuit = get_qiskit_circuit(circuit)

In [16]:
# p =Program.parse("(lambda (lambda (lambda (lambda (cnot (#(lambda (lambda (x (h $0 $1) $1))) $2 (x $0 $2)) $2 $1)))))")
# p =Program.parse("cnot")
p = Program.parse("(lambda $1)")
p

(lambda $1)

In [17]:
p.evaluate([3])([])

3

In [18]:
p.body.body.body.body.f.f.x.f.f.body.body.body.f.x.x

AttributeError: 'Index' object has no attribute 'body'

In [19]:
type(p)

dreamcoder.program.Abstraction

In [20]:
p.body.x.evaluate

AttributeError: 'Index' object has no attribute 'x'

In [22]:
ee = p.evaluate([1])
ee(no_op(2))(1)(0)

TypeError: 'int' object is not callable

In [21]:
import inspect

inspect.getsource(ee)

NameError: name 'ee' is not defined